In [ ]:
import json
import csv
import pandas as pd
import requests

### Importing related matches

In [ ]:
# import from Words Matter KG
url = 'https://raw.githubusercontent.com/cultural-ai/wordsmatter/main/related_matches/rm.json'
r = requests.get(url)
rms = r.json()

In [ ]:
# related matches from ODWN are in a separate file
with open('LODlit/bg/related_matches_odwn.json','r') as jf:
    rms_odwn = json.load(jf)

In [ ]:
# importing query terms with lemmas
with open('LODlit/query_terms.json','r') as jf:
    query_terms = json.load(jf)

In [ ]:
def get_lemma_by_term(query_term:str, lang:str) -> str:
    '''
    Getting a lemma of a query term
    lang: str, 'en' or 'nl'
    Returns str, 'not found' if lemma was not found
    '''
    
    return_lemma = 'not found'
    
    # importing query terms with lemmas
    # change path to GitHub
    
    with open('LODlit/query_terms.json','r') as jf:
        query_terms = json.load(jf)
        
    for lemma, qt in query_terms[lang].items():
        if query_term in qt:
            return_lemma = lemma
            
    return return_lemma

### How many query terms are associated with related matches?

In [ ]:
# importing bg info
with open('LODlit/bg/background_info_bows.json','r') as jf:
    bg_info = json.load(jf)

In [ ]:
# EN 
no_rm = []
for query_term, bows in bg_info["en"].items():
    if bows.get("wm") != None and len(bows) == 1:
        no_rm.append(query_term)

In [ ]:
len(no_rm)

In [ ]:
set([get_lemma_by_term(term,'en') for term in no_rm])

In [ ]:
# NL 
no_rm_nl = []
for query_term, bows in bg_info["nl"].items():
    if bows.get("wm") != None and len(bows) == 1:
        no_rm_nl.append(query_term)

In [ ]:
len(no_rm_nl)

In [ ]:
set([get_lemma_by_term(term,'nl') for term in no_rm_nl])

### Importing search results and generating a subset with related matches for every resource

In [ ]:
def generate_rm_subset(resource_data:dict, rm_data:dict, resource:str, lang:str) -> dict:
    '''
    Retrieves info about entities that are related matches
    resource_data: dict, search results in a resource
    rm_data: dict, info about related matches ('rm.json')
    resource: str, 'wikidata', 'aat', 'pwn', 'odwn'
    lang: str, 'en' or 'nl'
    Returns dict 
    '''
    results = {}

    for query_term, value in resource_data.items():
        
        hits_by_query_term = []
    
        for rm_info in rm_data.values():
            
            if rm_info['lang'] == lang and query_term in rm_info['query_terms']:
                for hit in value:
                    
                    # checking resource
                    
                    if resource == 'wikidata':
                        
                        if hit.get('QID') == rm_info['related_matches']['wikidata'][0]:
                            hits_by_query_term.append(hit)
                            
                    if resource == 'aat':
                        
                        if hit.get('aat_uri') == rm_info['related_matches']['aat'][0]:
                            hits_by_query_term.append(hit)
                            
                    if resource == 'pwn':
                        
                        if hit.get('synset_id') in rm_info['related_matches']['pwn']:
                            hits_by_query_term.append(hit)
                            
                    # ODWN is handled separately
                            
        results[query_term] = hits_by_query_term
        
    return results

#### Wikidata EN

In [ ]:
# this file is gzipped on GitHub with the prefix "gzip_"
with open('results_clean_en.json','r') as jf:
    wd_en = json.load(jf)

In [ ]:
rm_wd_en = generate_rm_subset(wd_en,rms,"wikidata","en")

In [ ]:
with open('LODlit/rm/rm_wd_en.json', 'w') as jf:
    json.dump(rm_wd_en, jf)

#### Wikidata NL

In [ ]:
# this file is gzipped on GitHub with the prefix "gzip_"
with open('results_clean_nl.json','r') as jf:
    wd_nl = json.load(jf)

In [ ]:
rm_wd_nl = generate_rm_subset(wd_nl,rms,"wikidata","nl")

In [ ]:
with open('LODlit/rm/rm_wd_nl.json', 'w') as jf:
    json.dump(rm_wd_nl, jf)

#### AAT EN

In [ ]:
with open('LODlit/AAT/aat_query_results_en.json','r') as jf:
    aat_en = json.load(jf)

In [ ]:
rm_aat_en = generate_rm_subset(aat_en,rms,"aat","en")

In [ ]:
with open('LODlit/rm/rm_aat_en.json', 'w') as jf:
    json.dump(rm_aat_en, jf)

#### AAT NL

In [ ]:
with open('LODlit/AAT/aat_query_results_nl.json','r') as jf:
    aat_nl = json.load(jf)

In [ ]:
rm_aat_nl = generate_rm_subset(aat_nl,rms,"aat","nl")

In [ ]:
with open('LODlit/rm/rm_aat_nl.json', 'w') as jf:
    json.dump(rm_aat_nl, jf)

#### PWN

In [ ]:
with open('LODlit/PWN/pwn31_query_results.json','r') as jf:
    pwn = json.load(jf)

In [ ]:
rm_pwn = generate_rm_subset(pwn,rms,"pwn","en")

In [ ]:
with open('LODlit/rm/rm_pwn.json', 'w') as jf:
    json.dump(rm_pwn, jf)

#### ODWN

In [ ]:
with open('LODlit/ODWN/odwn_query_results.json','r') as jf:
    odwn = json.load(jf)

In [ ]:
rm_odwn = {}

for query_term, value in odwn.items():
    
    hits_by_query_term = []
    
    for rm_info in rms_odwn.values():
        if query_term in rm_info['query_terms']:
            
            for hit in value:
                # all related matches in ODWN have LE_id
                if hit.get('le_id') in rm_info['odwn_le']:
                    hits_by_query_term.append(hit)
                    
    rm_odwn[query_term] = hits_by_query_term

In [ ]:
with open('LODlit/rm/rm_odwn.json', 'w') as jf:
    json.dump(rm_odwn, jf)

### N unique related matches per resource

In [ ]:
quids = []

for rm_info in rm_wd_en.values():
    quids.extend([hit['QID'] for hit in rm_info])
    
print("Wikidata EN:",len(set(quids)))

In [ ]:
quids = []

for rm_info in rm_wd_nl.values():
    quids.extend([hit['QID'] for hit in rm_info])
    
print("Wikidata NL:",len(set(quids)))

In [ ]:
aat_uris = []

for rm_info in rm_aat_en.values():
    aat_uris.extend([hit['aat_uri'] for hit in rm_info])
    
print("AAT EN:",len(set(aat_uris)))

In [ ]:
aat_uris = []

for rm_info in rm_aat_nl.values():
    aat_uris.extend([hit['aat_uri'] for hit in rm_info])
    
print("AAT NL:",len(set(aat_uris)))

In [ ]:
pwn_synsets = []

for rm_info in rm_pwn.values():
    pwn_synsets.extend([hit['synset_id'] for hit in rm_info])
    
print("PWN:",len(set(pwn_synsets)))

In [ ]:
odwn_le = []

for rm_info in rm_odwn.values():
    odwn_le.extend([hit['le_id'] for hit in rm_info])
    
print("ODWN:",len(set(odwn_le)))

### Overview by properties

In [ ]:
import statistics
from statistics import Counter

In [ ]:
all_occurences = []
for query_term, rm_info in rm_wd_en.items():
    for hit in rm_info:
        all_occurences.append(hit["found_in"])

In [ ]:
Counter(all_occurences)

In [ ]:
total_hits = 0
for query_term, rm_info in rm_wd_en.items():
    total_hits += len(rm_info)

In [ ]:
total_hits

In [ ]:
for query_term, rm_info in rm_wd_en.items():
    for hit in rm_info:
        if hit["found_in"] == 'description':
            print(query_term)

In [ ]:
# NL wikidata
all_occurences_nl = []
for query_term, rm_info in rm_wd_nl.items():
    for hit in rm_info:
        all_occurences_nl.append(hit["found_in"])

In [ ]:
Counter(all_occurences_nl)

In [ ]:
# EN AAT
all_occurences_aat_en = []
for query_term, rm_info in rm_aat_en.items():
    for hit in rm_info:
        all_occurences_aat_en.append(hit["found_in"])
Counter(all_occurences_aat_en)

In [ ]:
len(all_occurences_aat_en)

In [ ]:
# NL AAT
all_occurences_aat_nl = []
for query_term, rm_info in rm_aat_nl.items():
    for hit in rm_info:
        all_occurences_aat_nl.append(hit["found_in"])
Counter(all_occurences_aat_nl)

In [ ]:
len(all_occurences_aat_nl)

In [ ]:
# PWN
all_occurences_pwn = []
for query_term, rm_info in rm_pwn.items():
    for hit in rm_info:
        all_occurences_pwn.append(hit["found_in"])
Counter(all_occurences_pwn)

In [ ]:
len(all_occurences_pwn)

In [ ]:
# ODWN
all_occurences_odwn = []
for query_term, rm_info in rm_odwn.items():
    for hit in rm_info:
        all_occurences_odwn.append(hit["found_in"])
Counter(all_occurences_odwn)

In [ ]:
len(all_occurences_odwn)

In [ ]:
sum(Counter(all_occurences_odwn).values())

In [ ]:
# make an overview for all resources
# EN
with open('rm_stats_en.csv','w') as csv_file:
    
    writer = csv.writer(csv_file)
    header = ["query_term","lang","wd_pref","wd_aliases","wd_descr","wd_total",\
              "aat_pref","aat_alt","aat_scopeNote","aat_total",\
              "pwn_le","pwn_def","pwn_examples","pwn_total","total_hits"]
    writer.writerow(header)
    
    for query_term in rm_wd_en.keys():
        
        # wikidata
        wd_hits = Counter([hit["found_in"] for hit in rm_wd_en[query_term]])
        wd_pref = wd_hits.get('prefLabel')
        wd_aliases = wd_hits.get('aliases')
        wd_descr = wd_hits.get('description')
        wd_total = sum(Counter(wd_hits).values())
        
        # aat
        aat_hits = Counter([hit["found_in"] for hit in rm_aat_en[query_term]])
        aat_pref = aat_hits.get('prefLabel')
        aat_alt = aat_hits.get('altLabel')
        aat_scopeNote = aat_hits.get('scopeNote')
        aat_total = sum(Counter(aat_hits).values())
        
        # pwn
        pwn_hits = Counter([hit["found_in"] for hit in rm_pwn[query_term]])
        pwn_le = pwn_hits.get('lemmata')
        pwn_def = pwn_hits.get('definition')
        pwn_examples = pwn_hits.get('examples')
        pwn_total = sum(Counter(pwn_hits).values())
        
        total_hits = wd_total + aat_total + pwn_total
        
        data = [query_term,"en",wd_pref,wd_aliases,wd_descr,wd_total,aat_pref,aat_alt,aat_scopeNote,aat_total,\
               pwn_le,pwn_def,pwn_examples,pwn_total,total_hits]
        
        writer.writerow(data)

In [ ]:
# which terms and lemmas have no hits
# EN

no_hits = []

for query_term in rm_wd_en.keys():
        
    # wikidata
    wd_hits = Counter([hit["found_in"] for hit in rm_wd_en[query_term]])
    wd_pref = wd_hits.get('prefLabel')
    wd_aliases = wd_hits.get('aliases')
    wd_descr = wd_hits.get('description')
    wd_total = sum(Counter(wd_hits).values())

    # aat
    aat_hits = Counter([hit["found_in"] for hit in rm_aat_en[query_term]])
    aat_pref = aat_hits.get('prefLabel')
    aat_alt = aat_hits.get('altLabel')
    aat_scopeNote = aat_hits.get('scopeNote')
    aat_total = sum(Counter(aat_hits).values())

    # pwn
    pwn_hits = Counter([hit["found_in"] for hit in rm_pwn[query_term]])
    pwn_le = pwn_hits.get('lemmata')
    pwn_def = pwn_hits.get('definition')
    pwn_examples = pwn_hits.get('examples')
    pwn_total = sum(Counter(pwn_hits).values())

    total_hits = wd_total + aat_total + pwn_total

    if total_hits == 0:
        no_hits.append(query_term)

In [ ]:
len(no_hits)

In [ ]:
for lemma, wordforms in query_terms['en'].items():
    k = 0
    for w in wordforms:
        if w in no_hits:
            k += 1
            
    if k == len(wordforms):
        print(lemma)

In [ ]:
# NL
with open('rm_stats_nl.csv','w') as csv_file:
    
    writer = csv.writer(csv_file)
    header = ["query_term","lang","wd_pref","wd_aliases","wd_descr","wd_total",\
              "aat_pref","aat_alt","aat_scopeNote","aat_total",\
              "odwn_le","odwn_sense_examples","odwn_sense_definition","odwn_total","total_hits"]
    writer.writerow(header)
    
    for query_term in rm_wd_nl.keys():
        
        # wikidata
        wd_hits = Counter([hit["found_in"] for hit in rm_wd_nl[query_term]])
        wd_pref = wd_hits.get('prefLabel')
        wd_aliases = wd_hits.get('aliases')
        wd_descr = wd_hits.get('description')
        wd_total = sum(Counter(wd_hits).values())
        
        # aat
        aat_hits = Counter([hit["found_in"] for hit in rm_aat_nl[query_term]])
        aat_pref = aat_hits.get('prefLabel')
        aat_alt = aat_hits.get('altLabel')
        aat_scopeNote = aat_hits.get('scopeNote')
        aat_total = sum(Counter(aat_hits).values())
        
        # pwn
        odwn_hits = Counter([hit["found_in"] for hit in rm_odwn[query_term]])
        odwn_le = odwn_hits.get('le')
        odwn_sense_ex = odwn_hits.get('sense_examples')
        odwn_sense_def = odwn_hits.get('sense_definition')
        odwn_total = sum(Counter(odwn_hits).values())
        
        total_hits = wd_total + aat_total + odwn_total
        
        data = [query_term,"nl",wd_pref,wd_aliases,wd_descr,wd_total,aat_pref,aat_alt,aat_scopeNote,aat_total,\
               odwn_le,odwn_sense_ex,odwn_sense_def,odwn_total,total_hits]
        
        writer.writerow(data)

In [ ]:
# which terms and lemmas have no hits
# NL
no_hits = []

for query_term in rm_wd_nl.keys():
        
    # wikidata
    wd_hits = Counter([hit["found_in"] for hit in rm_wd_nl[query_term]])
    wd_pref = wd_hits.get('prefLabel')
    wd_aliases = wd_hits.get('aliases')
    wd_descr = wd_hits.get('description')
    wd_total = sum(Counter(wd_hits).values())

    # aat
    aat_hits = Counter([hit["found_in"] for hit in rm_aat_nl[query_term]])
    aat_pref = aat_hits.get('prefLabel')
    aat_alt = aat_hits.get('altLabel')
    aat_scopeNote = aat_hits.get('scopeNote')
    aat_total = sum(Counter(aat_hits).values())

    # pwn
    odwn_hits = Counter([hit["found_in"] for hit in rm_odwn[query_term]])
    odwn_le = odwn_hits.get('le')
    odwn_sense_ex = odwn_hits.get('sense_examples')
    odwn_sense_def = odwn_hits.get('sense_definition')
    odwn_total = sum(Counter(odwn_hits).values())

    total_hits = wd_total + aat_total + odwn_total
    
    if total_hits == 0:
        no_hits.append(query_term)

In [ ]:
len(no_hits)

In [ ]:
for lemma, wordforms in query_terms['nl'].items():
    k = 0
    for w in wordforms:
        if w in no_hits:
            k += 1
            
    if k == len(wordforms):
        print(lemma)

### Overview by lemma

#### EN

In [ ]:
stats_en = pd.read_csv('LODlit/rm/rm_stats_en.csv')

In [ ]:
stats_en.fillna(0, inplace=True)

In [ ]:
# get the lemmas column for each query term EN
lemmas = []
for row in stats_en.iterrows():
    for lemma, wordforms in query_terms['en'].items():
        if row[1]['query_term'] in wordforms:
            lemmas.append(lemma)

In [ ]:
stats_en.insert(0,"lemma",lemmas)

In [ ]:
# lemmas EN

with open('rm_stats_by_lemma_en.csv','w') as csv_file:
    
    writer = csv.writer(csv_file)
    header = ["lemma","lang","wd_pref","wd_aliases","wd_descr","wd_total",\
              "aat_pref","aat_alt","aat_scopeNote","aat_total",\
              "pwn_le","pwn_def","pwn_examples","pwn_total","total_hits"]
    writer.writerow(header)
    
    
    for lemma_group in stats_en.groupby("lemma"):
        data = [lemma_group[0],"en",sum(lemma_group[1]['wd_pref']),sum(lemma_group[1]['wd_aliases']),\
              sum(lemma_group[1]['wd_descr']),sum(lemma_group[1]['wd_total']),\
              sum(lemma_group[1]['aat_pref']),sum(lemma_group[1]['aat_alt']),\
              sum(lemma_group[1]['aat_scopeNote']),sum(lemma_group[1]['aat_total']),\
              sum(lemma_group[1]['pwn_le']),sum(lemma_group[1]['pwn_def']),\
              sum(lemma_group[1]['pwn_examples']),sum(lemma_group[1]['pwn_total']),\
              sum(lemma_group[1]['total_hits'])]
        
        writer.writerow(data)

#### NL

In [ ]:
stats_nl = pd.read_csv('LODlit/rm/rm_stats_nl.csv')

In [ ]:
stats_nl.fillna(0, inplace=True)

In [ ]:
# get the lemmas column for each query term EN
lemmas = []
for row in stats_nl.iterrows():
    for lemma, wordforms in query_terms['nl'].items():
        if row[1]['query_term'] in wordforms:
            lemmas.append(lemma)

In [ ]:
stats_nl.insert(0,"lemma",lemmas)

In [ ]:
# lemmas NL

with open('rm_stats_by_lemma_nl.csv','w') as csv_file:
    
    writer = csv.writer(csv_file)
    header = ["lemma","lang","wd_pref","wd_aliases","wd_descr","wd_total",\
              "aat_pref","aat_alt","aat_scopeNote","aat_total",\
              "odwn_le","odwn_sense_examples","odwn_sense_definition","odwn_total","total_hits"]
    writer.writerow(header)
    
    
    for lemma_group in stats_nl.groupby("lemma"):
        data = [lemma_group[0],"nl",sum(lemma_group[1]['wd_pref']),sum(lemma_group[1]['wd_aliases']),\
              sum(lemma_group[1]['wd_descr']),sum(lemma_group[1]['wd_total']),\
              sum(lemma_group[1]['aat_pref']),sum(lemma_group[1]['aat_alt']),\
              sum(lemma_group[1]['aat_scopeNote']),sum(lemma_group[1]['aat_total']),\
              sum(lemma_group[1]['odwn_le']),sum(lemma_group[1]['odwn_sense_examples']),\
              sum(lemma_group[1]['odwn_sense_definition']),sum(lemma_group[1]['odwn_total']),\
              sum(lemma_group[1]['total_hits'])]
        
        writer.writerow(data)

In [ ]:
### Generate a csv with related matches
# group by lemmas

In [ ]:
with open('rm_entities_all.csv','w') as csv_file:
    
    writer = csv.writer(csv_file)
    header = ["lemma","lang","resource","entity_id"]
    writer.writerow(header)
        
    # EN wikidata
    for query_term, rms in rm_wd_en.items():
        for rm in rms:
            data = [get_lemma_by_term(query_term,'en'),'en',"wikidata",rm['QID']]
            writer.writerow(data)
            
    # NL wikidata
    for query_term, rms in rm_wd_nl.items():
        for rm in rms:
            data = [get_lemma_by_term(query_term,'nl'),'nl',"wikidata",rm['QID']]
            writer.writerow(data)
            
    # EN AAT
    for query_term, rms in rm_aat_en.items():
        for rm in rms:
            data = [get_lemma_by_term(query_term,'en'),'en',"aat",rm['aat_uri']]
            writer.writerow(data)
    
    # NL AAT
    for query_term, rms in rm_aat_nl.items():
        for rm in rms:
            data = [get_lemma_by_term(query_term,'nl'),'nl',"aat",rm['aat_uri']]
            writer.writerow(data)
            
    # PWN
    for query_term, rms in rm_pwn.items():
        for rm in rms:
            data = [get_lemma_by_term(query_term,'en'),'en',"pwn",rm['synset_id']] 
            writer.writerow(data)
            
    # ODWN
    for query_term, rms in rm_odwn.items():
        for rm in rms:
            if rm['synset_id'] == '':
                data = [get_lemma_by_term(query_term,'nl'),'nl',"odwn",rm['sense_id']]
                writer.writerow(data)
            else:
                data = [get_lemma_by_term(query_term,'nl'),'nl',"odwn",rm['synset_id']]
                writer.writerow(data)


In [ ]:
all_entities = pd.read_csv("LODlit/rm/rm_entities_all.csv")

In [ ]:
all_entities.drop_duplicates(inplace=True)

In [ ]:
all_entities.to_csv("LODlit/rm/rm_entities_unique.csv")